# Run multiple robots for one day

Run an experiment with multiple robots exploring an environment during the course of one day. 

The specification of the experiment is fully determined by the exp/run of type nrobot1day. This exp/run will refer to other exp/runs to specify the environment, the estimator and the scoring function. It will also list the names of the robots, their policies and other information. 

In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
from pprint import pprint

from wbf_simulate import run_nrobot1day

In [ ]:
# the experiment for the robot run
experiment = "nrobot1day"
# run = "two-random-robots"
# run = "two-communicating-robots"
# run = "mrmr-3random"
# run = "mrmr-3lawnmowers"
run = "mrmr-1pioneer-2contractors"
exp = Config().get_experiment(experiment, run)
pprint(exp)

In [ ]:
run_nrobot1day(exp)

AssertionError: 